In [1]:
# Add ldcpy root to system path
import sys

import astropy

sys.path.insert(0, '../../../')

# Import ldcpy package
# Autoreloads package everytime the package is called, so changes to code will be reflected in the notebook if the above sys.path.insert(...) line is uncommented.
%load_ext autoreload
%autoreload 2

# suppress all of the divide by zero warnings
import warnings

warnings.filterwarnings("ignore")

import ldcpy

# display the plots in this notebook
%matplotlib inline
import numpy as np


In [2]:
# col_t contains 3D T data (here we specify the chunk to be a single timeslice)
col_t = ldcpy.open_datasets(
    "cam-fv",
    ["T"],
    [
        "../../../data/cam-fv/cam-fv.T.3months.nc",
        "../../../data/cam-fv/c.fpzip.cam-fv.T.3months.nc",
    ],
    ["orig", "comp"],
    chunks={"time": 1},
)


# col_ts is a collection containing TS data
col_ts = ldcpy.open_datasets(
    "cam-fv",
    ["TS"],
    [
        "../../../data/cam-fv/orig.TS.100days.nc",
        "../../../data/cam-fv/zfp1.0.TS.100days.nc",
        "../../../data/cam-fv/zfp1e-1.TS.100days.nc",
    ],
    ["orig", "zfpA1.0", "zfpA1e-1"],
)

dataset size in GB 0.04

dataset size in GB 0.07



In [7]:
T_orig = col_t["T"].sel(collection = "orig").isel(time = 0).data.compute()
print(T_orig.shape)
TS_orig = col_ts["TS"].sel(collection = "orig").isel(time = 0).data.compute()
print(TS_orig.shape)

(30, 192, 288)
(192, 288)


In [8]:
#With 3d
#shift in lon
y_lon_3d = T_orig[:,:, 1:]
x_lon_3d = T_orig[:, :,:-1]
print(y_lon_3d.shape)
print(x_lon_3d.shape)
y_lon_3d_r = np.ravel(y_lon_3d)
x_lon_3d_r = np.ravel(x_lon_3d)
np.corrcoef(x_lon_3d_r, y_lon_3d_r)[0,1]

(30, 192, 287)
(30, 192, 287)


0.9998740566679775

In [9]:
#With 3d
#shift in lat
y_lat_3d = T_orig[:,1:,:]
x_lat_3d = T_orig[:, :-1,:]
print(y_lat_3d.shape)
print(x_lat_3d.shape)
y_lat_3d_r = np.ravel(y_lat_3d)
x_lat_3d_r = np.ravel(x_lat_3d)
np.corrcoef(x_lat_3d_r, y_lat_3d_r)[0,1]

(30, 191, 288)
(30, 191, 288)


0.9996481022999253

In [10]:
#With 3d
#shift in lev 
y_lev_3d = T_orig[1:,:,:]
x_lev_3d = T_orig[:-1, :,:]
print(y_lev_3d.shape)
print(x_lev_3d.shape)
y_lev_3d_r = np.ravel(y_lev_3d)
x_lev_3d_r = np.ravel(x_lev_3d)
np.corrcoef(x_lev_3d_r, y_lev_3d_r)[0,1]

(29, 192, 288)
(29, 192, 288)


0.9867696643216919

In [223]:
idx = 0
y = np.roll(T_orig, -1, axis = idx )
x = T_orig
y_r = np.ravel(y)
x_r = np.ravel(x)
aa = np.corrcoef(x_r, y_r)[0,1]
print(aa)
print(x.shape)
print(y.shape)

0.9508834044953541
(30, 192, 288)
(30, 192, 288)


In [228]:
yd =  np.delete(y, 29,axis = idx)
xd = np.delete(x, 29, axis = idx)
yd_r = np.ravel(yd)
xd_r = np.ravel(xd)
aa = np.corrcoef(xd_r, yd_r)[0,1]
print(aa)

0.9867696643216919


14.280191


In [326]:
T_ds = col_t["T"].sel(collection = "orig").isel(time = 0).compute()
T_ds_calcs = ldcpy.Datasetcalcs(T_ds, "cam-fv", ["lat", 'lon'])
#T_ds.attrs
TS_ds = col_ts["TS"].sel(collection = "orig").isel(time = 0).compute()
TS_ds_calcs = ldcpy.Datasetcalcs(TS_ds, "cam-fv", ["lat", 'lon'])

In [327]:
#aa = T_ds_calcs.get_calc("mean").compute().data
#print(aa)
a1 = T_ds_calcs.get_single_calc("lat_autocorr")
a2 = T_ds_calcs.get_single_calc("lon_autocorr")
a3 = T_ds_calcs.get_single_calc("lev_autocorr")
print("T:",a1, a2, a3)
b1 = TS_ds_calcs.get_single_calc("lat_autocorr")
b2 = TS_ds_calcs.get_single_calc("lon_autocorr")
#b3 = TS_ds_calcs.get_calc("lev_autocorr")
print("TS:",b1,b2)

T: 0.9996481022999253 0.9998742881758094 0.9867696643216919
TS: 0.9939178070623784 0.9968008894669186


In [328]:
et = T_ds_calcs.get_single_calc("entropy")
ets = TS_ds_calcs.get_single_calc("entropy")
print(et)
print(ets)

0.39041748046875
0.41472258391203703


In [208]:
d = T_ds.dims
print(d)
if 'lat' in d:
    print('yes')


('lev', 'lat', 'lon')
yes


In [115]:
dims = T_ds.dims
print(dims)
dims.index('lat')

('lev', 'lat', 'lon')


1

In [234]:
T_ds.sizes["lev"]

30

[[3.399926  3.399926  3.399926  ... 3.399926  3.399926  3.399926 ]
 [3.3999171 3.3999174 3.3999188 ... 3.399915  3.3999155 3.399917 ]
 [3.3998876 3.39989   3.3998923 ... 3.3998806 3.3998828 3.3998854]
 ...
 [3.3956032 3.3955994 3.3955967 ... 3.395615  3.3956108 3.3956065]
 [3.3958275 3.3958273 3.3958268 ... 3.3958294 3.3958285 3.3958275]
 [3.3960855 3.3960855 3.3960855 ... 3.3960855 3.3960855 3.3960855]]


In [11]:
#With 2d
#shift in lon
y_lon_2d = TS_orig[:, 1:]
x_lon_2d = TS_orig[:,:-1]
print(y_lon_2d.shape)
print(x_lon_2d.shape)
y_lon_2d_r = np.ravel(y_lon_2d)
x_lon_2d_r = np.ravel(x_lon_2d)
np.corrcoef(x_lon_2d_r, y_lon_2d_r)[0,1]

(192, 287)
(192, 287)


0.996804517222752

In [12]:
#With dd
#shift in lat
y_lat_2d = TS_orig[1:,:]
x_lat_2d = TS_orig[:-1,:]
print(y_lat_2d.shape)
print(x_lat_2d.shape)
y_lat_2d_r = np.ravel(y_lat_2d)
x_lat_2d_r = np.ravel(x_lat_2d)
np.corrcoef(x_lat_2d_r, y_lat_2d_r)[0,1]

(191, 288)
(191, 288)


0.9939178070623784

In [170]:
y_lat = TS_orig[1:,:]
x_lat = TS_orig[:-1,:]
test_y = np.roll(TS_orig, -1, axis =0 )
test_x = TS_orig
test_x_r = np.ravel(test_x)
test_y_r = np.ravel(test_y)
np.corrcoef(test_x_r, test_y_r)[0,1]

0.9930221542020569

In [178]:
y_lat = T_orig[:,1:,:]
x_lat = T_orig[:,:-1,:]
test_y = np.roll(T_orig, -1, axis =1 )
test_x = T_orig
test_x_r = np.ravel(test_x)
test_y_r = np.ravel(test_y)
np.corrcoef(test_y_r, test_x_r)[0,1]

0.997681332316324

In [163]:
test.shape

(192, 288)

In [164]:
y_lon_2d.shape

(192, 287)

In [168]:
print(test_y[0,0])
print(test_x[1,0])
test_y[0:3, 0]

241.89694
241.89694


array([241.89694, 242.16418, 243.27536], dtype=float32)

In [268]:
TS_orig.shape

(192, 288)

In [269]:
import scipy.stats
a = scipy.stats.entropy(TS_orig)
print(a.shape)


(288,)


In [270]:
import gzip

In [289]:
cc = gzip.compress(TS_orig)

In [290]:
dd = gzip.decompress(cc)

In [296]:
ld = len(dd)
ld

221184

In [297]:
lc = len(cc)
lc

170898

In [294]:
lc/ld

0.7726508246527778

b'\xd15rC\xf3\x1dsC\x02)sC\xb6/sC\xad5rC\xc6?rC\x855rC\xb9IrC\xf59rC\x8f*rC\xcd!sC\xcd\x98rC\x9d.rC\xe81rC/7rCh2sC\xc1\x94rC0ZsC\xfe1rC\xc3\x14sC\t>rCa2rC\x7f/sC\xc0:sC\xe38rC\xfc\x1fsC\xd1+rCE7rCG*rC\x05\xd8qC\xf2XsC\xa4\x11sC\x87#rC}&rCO\x93rCe\xedrC\xbdHsC\xb0\xe4qC72rC<&sC\xcc2sC[GsCb9rC\xc8\rsC\xa39rC\xeb/rC\x855rC)PsC\x89/rC\x1e\rsC\x1f%rC\xf0/rCH\xderC,*sC\x876rC|;sC\x0f$sC\xc9\xddqCz*sC\x95%sC?4rC~#sC\xcb2rCt.rC]\x9brCT\x0esC\x8f:rC\x9f1rC\x1c1rC\xe91rC\xe7\x11sC,/rCW3rC\xef\x0csCi3rC\xba\x95rC\x160rC\x112rC\xe6(sCB\x13sC\xb5:sC\x99\x8drC\\)sC\x84\'sCy\x94rC!\x05sC\xa9@rCl\x00sC\xa4/sC\x135rC\x9f\x0fsC\n\x13sC<+rCr,rCh-rC\x85\rsCd\x08sCMPsC\xe8AsCg8sCO"sC\xd2\xdcrC)%rC\rzrCh\xe5qC\xd8-sC\x84"rCi)rC=\xe2rC\xb88rCP&rC\xd5\rsC )sC40sC\x960rC\xe9\x99rCN\x0csC\xaa\rsC\xc3>sC\x12\rsC\x06)sC\xe7\x15sC\xbf1sC\x06\x89rC\x85\x12sC9<rC\xca\x1asC \x04sC\xaa3rC+\x91rC\x03/sCh\x00sC 0rC!ZsC\x95\x11sC\x0c3rC\xa6\xefrC:8rCx9rC\xb6*rC\\*rC\xbf3rCTQsC\xbf\xc7rC?\xfbrC\x81.rCG*rC\xb8FsC$$sC\xd2&r